# Create SGS model assemblies

This notebook creates and exports wrapper models to be used in OpenFOAM simulations. Each wrapper model hides the single MLP models for diffusive and convective fluxes for one species. The wrapper model also contains constants to scale and rescale model inputs and outputs.

## Dependencies

Notebooks (links work only if the Docker container is running)

- [create_sgs_models.ipynb](http://127.0.0.1:8888/notebooks/create_sgs_models.ipynb)
- [reduce_training_data.ipynb](http://127.0.0.1:8888/notebooks/reduce_training_data.ipynb)

## Model assemblies

In [1]:
import helper_module as hm
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

data_path = "../data/sgs_data/"
output_path_model = "../output/models/"
torch.set_default_tensor_type(torch.DoubleTensor)

In [2]:
reactions = ["phys", "decay", "single", "cons"]
data_files = {}
for r in reactions:
    path = data_path + "{:s}_red.csv".format(r)
    data_files[r] = pd.read_csv(path, header=0)
    
model_dict = {
    "n_inputs" : 2,
    "n_outputs" : 1,
    "n_layers" : 6,
    "n_neurons" : 40,
    "activation" : torch.nn.functional.relu,
    "batch_norm" : False
}

In [3]:
class A_Model(torch.nn.Module):
    def __init__(self, modelGrad_s, modelGrad_f, model_f, X_max, X_min, y_max, y_min):
        super().__init__()
        self.modelGrad_s = modelGrad_s
        self.modelGrad_f = modelGrad_f
        self.model_f = model_f
        self.X_max = torch.tensor(X_max, dtype=torch.float64)
        self.X_min = torch.tensor(X_min, dtype=torch.float64)
        self.grad_s_max = torch.tensor(y_max[0], dtype=torch.float64)
        self.grad_s_min = torch.tensor(y_min[0], dtype=torch.float64)
        self.grad_f_max = torch.tensor(y_max[1], dtype=torch.float64)
        self.grad_f_min = torch.tensor(y_min[1], dtype=torch.float64)
        self.f_max = torch.tensor(y_max[2], dtype=torch.float64)
        self.f_min = torch.tensor(y_min[2], dtype=torch.float64)
        
    def forward(self, Xorg):
        X = Xorg.clone()
        X.requires_grad_(False)
        X = (X - self.X_min) / (self.X_max - self.X_min)
        grad_s = self.modelGrad_s(X)
        grad_f = self.modelGrad_f(X)
        f = self.model_f(X)
        grad_s = grad_s * (self.grad_s_max - self.grad_s_min) + self.grad_s_min
        grad_f = grad_f * (self.grad_f_max - self.grad_f_min) + self.grad_f_min
        f = f * (self.f_max - self.f_min) + self.f_min
        return torch.cat((grad_s, grad_f, f), 1)

    
class BPS_Model(torch.nn.Module):
    def __init__(self, modelGrad_f, model_f, X_max, X_min, y_max, y_min):
        super().__init__()
        self.modelGrad_f = modelGrad_f
        self.model_f = model_f
        self.X_max = torch.tensor(X_max, dtype=torch.float64)
        self.X_min = torch.tensor(X_min, dtype=torch.float64)
        self.grad_f_max = torch.tensor(y_max[0], dtype=torch.float64)
        self.grad_f_min = torch.tensor(y_min[0], dtype=torch.float64)
        self.f_max = torch.tensor(y_max[1], dtype=torch.float64)
        self.f_min = torch.tensor(y_min[1], dtype=torch.float64)
        
    def forward(self, Xorg):
        X = Xorg.clone()
        X.requires_grad_(False)
        X = (X - self.X_min) / (self.X_max - self.X_min)
        grad_f = self.modelGrad_f(X)
        f = self.model_f(X)
        grad_f = grad_f * (self.grad_f_max - self.grad_f_min) + self.grad_f_min
        f = f * (self.f_max - self.f_min) + self.f_min
        return torch.cat((grad_f, f), 1)
    
    
class R_Model(torch.nn.Module):
    def __init__(self, model_r, X_max, X_min, y_max, y_min):
        super().__init__()
        self.model_r = model_r
        self.X_max = torch.tensor(X_max, dtype=torch.float64)
        self.X_min = torch.tensor(X_min, dtype=torch.float64)
        self.r_max = torch.tensor(y_max[0], dtype=torch.float64)
        self.r_min = torch.tensor(y_min[0], dtype=torch.float64)
        
    def forward(self, Xorg):
        X = Xorg.clone()
        X.requires_grad_(False)
        X = (X - self.X_min) / (self.X_max - self.X_min)
        r = self.model_r(X)
        r = r * (self.r_max - self.r_min) + self.r_min
        return r.squeeze()

## Physisorption

In [4]:
data = data_files["phys"]
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X01 = scaler_X.fit_transform(data[["dist", "A_av"]].values)
y01 = scaler_y.fit_transform(data[["gradA_s", "gradA_f", "A_f"]].values)
print("feature min: ", scaler_X.data_min_)
print("feature max: ", scaler_X.data_max_)
print("label min: ", scaler_y.data_min_)
print("label max: ", scaler_y.data_max_)

feature min:  [ 0.00014647  0.0190299 ]
feature max:  [ 0.02410139  0.99999401]
label min:  [ -1.35246401e+03  -1.29050920e+03   4.91842585e-35]
label max:  [ -8.16929972e-02  -5.09789595e-30   9.99988034e-01]


In [5]:
model_dict["n_inputs"] = 2

modelGrad_s = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "phys_gradA_s.pt"
modelGrad_s.load_state_dict(torch.load(model_path))
modelGrad_s.eval()

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "phys_gradA_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "phys_A_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_A_model = A_Model(modelGrad_s, modelGrad_f, model_f,
                       scaler_X.data_max_, scaler_X.data_min_,
                       scaler_y.data_max_, scaler_y.data_min_)

X = data[["dist", "A_av"]].values
model_trace = torch.jit.trace(full_A_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "A_model_phys.pt")

In [6]:
# sanity test
A_model_phys = torch.jit.load(output_path_model + "A_model_phys.pt")
X = data[["dist", "A_av"]].values
y = data[["gradA_s", "gradA_f", "A_f"]].values
ym = A_model_phys(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[  3.21585444e+01   6.45445141e+01   1.17640193e-02]


## Decay reaction

In [7]:
data = data_files["decay"]
scaler_X_A = MinMaxScaler()
scaler_X_P = MinMaxScaler()
scaler_y_A = MinMaxScaler()
scaler_y_P = MinMaxScaler()
X01_A = scaler_X_A.fit_transform(data[["dist", "A_av"]].values)
y01_A = scaler_y_A.fit_transform(data[["gradA_s", "gradA_f", "A_f"]])
X01_P = scaler_X_P.fit_transform(data[["dist", "A_av", "P_av"]].values)
y01_P = scaler_y_P.fit_transform(data[["gradP_f", "P_f"]])
print("feature A min: ", scaler_X_A.data_min_)
print("feature A max: ", scaler_X_A.data_max_)
print("feature P min: ", scaler_X_P.data_min_)
print("feature P max: ", scaler_X_P.data_max_)
print("label A min: ", scaler_y_A.data_min_)
print("label A max: ", scaler_y_A.data_max_)
print("label P min: ", scaler_y_P.data_min_)
print("label P max: ", scaler_y_P.data_max_)

feature A min:  [ 0.00014647  0.01901263]
feature A max:  [ 0.02410139  0.99720645]
feature P min:  [  1.46467255e-04   1.90126266e-02   3.33921849e-05]
feature P max:  [ 0.02410139  0.99720645  0.75023927]
label A min:  [ -1.35586926e+03  -1.29241793e+03   4.90573895e-35]
label A max:  [ -3.81287791e+01  -5.08476988e-30   9.94569296e-01]
label P min:  [ -1.38447926e+01   1.28078564e-37]
label P max:  [ -1.32709894e-32   7.50232730e-01]


In [8]:
model_dict["n_inputs"] = 2

modelGrad_s = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "decay_gradA_s.pt"
modelGrad_s.load_state_dict(torch.load(model_path))
modelGrad_s.eval()

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "decay_gradA_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "decay_A_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_A_model = A_Model(modelGrad_s, modelGrad_f, model_f,
                       scaler_X_A.data_max_, scaler_X_A.data_min_,
                       scaler_y_A.data_max_, scaler_y_A.data_min_)

X = data[["dist", "A_av"]].values
model_trace = torch.jit.trace(full_A_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "A_model_decay.pt")

In [9]:
# sanity test
A_model_decay = torch.jit.load(output_path_model + "A_model_decay.pt")
X = data[["dist", "A_av"]].values
y = data[["gradA_s", "gradA_f", "A_f"]].values
ym = A_model_decay(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[  5.19395579e+01   1.20408302e+02   2.06899039e-02]


In [10]:
model_dict["n_inputs"] = 3

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "decay_gradP_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "decay_P_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_P_model = BPS_Model(modelGrad_f, model_f,
                         scaler_X_P.data_max_, scaler_X_P.data_min_,
                         scaler_y_P.data_max_, scaler_y_P.data_min_)

X = data[["dist", "A_av", "P_av"]].values
model_trace = torch.jit.trace(full_P_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "P_model_decay.pt")

In [11]:
# sanity test
P_model_decay = torch.jit.load(output_path_model + "P_model_decay.pt")
X = data[["dist", "A_av", "P_av"]].values
y = data[["gradP_f", "P_f"]].values
ym = P_model_decay(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[ 0.6124376   0.00449567]


## Single reaction

In [12]:
data = data_files["single"]
scaler_X_AB = MinMaxScaler()
scaler_X_P = MinMaxScaler()
scaler_y_A = MinMaxScaler()
scaler_y_B = MinMaxScaler()
scaler_y_P = MinMaxScaler()
X01_AB = scaler_X_AB.fit_transform(data[["dist", "A_av", "B_av"]].values)
y01_A = scaler_y_A.fit_transform(data[["gradA_s", "gradA_f", "A_f"]])
y01_B = scaler_y_B.fit_transform(data[["gradB_f", "B_f"]])
X01_P = scaler_X_P.fit_transform(data[["dist", "A_av", "B_av", "P_av"]].values)
y01_P = scaler_y_P.fit_transform(data[["gradP_f", "P_f"]])
print("feature AB min: ", scaler_X_AB.data_min_)
print("feature AB max: ", scaler_X_AB.data_max_)
print("feature P min: ", scaler_X_P.data_min_)
print("feature P max: ", scaler_X_P.data_max_)
print("label A min: ", scaler_y_A.data_min_)
print("label A max: ", scaler_y_A.data_max_)
print("label B min: ", scaler_y_B.data_min_)
print("label B max: ", scaler_y_B.data_max_)
print("label P min: ", scaler_y_P.data_min_)
print("label P max: ", scaler_y_P.data_max_)

feature AB min:  [  1.46467255e-04   1.90126350e-02   4.70975454e-01]
feature AB max:  [ 0.02410139  0.99859163  0.99996662]
feature P min:  [  1.46467255e-04   1.90126350e-02   4.70975454e-01   3.33757974e-05]
feature P max:  [ 0.02410139  0.99859163  0.99996662  0.52902455]
label A min:  [ -1.35586645e+03  -1.29241664e+03   4.90573899e-35]
label A max:  [ -1.92225543e+01  -5.08476992e-30   9.97257107e-01]
label B min:  [-0.03779494  0.47097706]
label B max:  [ 13.20692147   1.        ]
label P min:  [ -1.32069215e+01   1.28078072e-37]
label P max:  [ 0.03779494  0.52902294]


In [13]:
model_dict["n_inputs"] = 3

modelGrad_s = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_gradA_s.pt"
modelGrad_s.load_state_dict(torch.load(model_path))
modelGrad_s.eval()

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_gradA_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_A_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_A_model = A_Model(modelGrad_s, modelGrad_f, model_f,
                       scaler_X_AB.data_max_, scaler_X_AB.data_min_,
                       scaler_y_A.data_max_, scaler_y_A.data_min_)

X = data[["dist", "A_av", "B_av"]].values
model_trace = torch.jit.trace(full_A_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "A_model_single.pt")

In [14]:
# sanity test
A_model_single = torch.jit.load(output_path_model + "A_model_single.pt")
X = data[["dist", "A_av", "B_av"]].values
y = data[["gradA_s", "gradA_f", "A_f"]].values
ym = A_model_single(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[  1.11384190e+02   1.27767044e+02   9.42756475e-03]


In [15]:
model_dict["n_inputs"] = 3

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_gradB_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_B_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_B_model = BPS_Model(modelGrad_f, model_f,
                         scaler_X_AB.data_max_, scaler_X_AB.data_min_,
                         scaler_y_B.data_max_, scaler_y_B.data_min_)

X = data[["dist", "A_av", "B_av"]].values
model_trace = torch.jit.trace(full_B_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "B_model_single.pt")

In [16]:
# sanity test
B_model_single = torch.jit.load(output_path_model + "B_model_single.pt")
X = data[["dist", "A_av", "B_av"]].values
y = data[["gradB_f", "B_f"]].values
ym = B_model_single(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[ 0.95675585  0.0074362 ]


In [17]:
model_dict["n_inputs"] = 4

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_gradP_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_P_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_P_model = BPS_Model(modelGrad_f, model_f,
                         scaler_X_P.data_max_, scaler_X_P.data_min_,
                         scaler_y_P.data_max_, scaler_y_P.data_min_)

X = data[["dist", "A_av", "B_av", "P_av"]].values
model_trace = torch.jit.trace(full_P_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "P_model_single.pt")

In [18]:
# sanity test
P_model_single = torch.jit.load(output_path_model + "P_model_single.pt")
X = data[["dist", "A_av", "B_av", "P_av"]].values
y = data[["gradP_f", "P_f"]].values
ym = P_model_single(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[ 0.61555782  0.00356164]


## Consecutive reaction

In [19]:
data = data_files["cons"]
scaler_X_ABP = MinMaxScaler()
scaler_X_ABPS = MinMaxScaler()
scaler_y_A = MinMaxScaler()
scaler_y_B = MinMaxScaler()
scaler_y_P = MinMaxScaler()
scaler_y_S = MinMaxScaler()
X01_ABP = scaler_X_ABP.fit_transform(data[["dist", "A_av", "B_av", "P_av"]].values)
y01_A = scaler_y_A.fit_transform(data[["gradA_s", "gradA_f", "A_f"]])
y01_B = scaler_y_B.fit_transform(data[["gradB_f", "B_f"]])
y01_P = scaler_y_P.fit_transform(data[["gradP_f", "P_f"]])
X01_ABPS = scaler_X_ABPS.fit_transform(data[["dist", "A_av", "B_av", "P_av", "S_av"]].values)
y01_S = scaler_y_S.fit_transform(data[["gradS_f", "S_f"]])
print("feature ABP min: ", scaler_X_ABP.data_min_)
print("feature ABP max: ", scaler_X_ABP.data_max_)
print("feature ABPS min: ", scaler_X_ABPS.data_min_)
print("feature ABPS max: ", scaler_X_ABPS.data_max_)
print("label A min: ", scaler_y_A.data_min_)
print("label A max: ", scaler_y_A.data_max_)
print("label B min: ", scaler_y_B.data_min_)
print("label B max: ", scaler_y_B.data_max_)
print("label P min: ", scaler_y_P.data_min_)
print("label P max: ", scaler_y_P.data_max_)
print("label S min: ", scaler_y_S.data_min_)
print("label S max: ", scaler_y_S.data_max_)

feature ABP min:  [  1.46467255e-04   1.90126183e-02   4.76073262e-01   3.33430253e-05]
feature ABP max:  [ 0.02410139  0.99723356  0.99996662  0.24925557]
feature ABPS min:  [  1.46467255e-04   1.90126183e-02   4.76073262e-01   3.33430253e-05
   3.27622541e-08]
feature ABPS max:  [ 0.02410139  0.99723356  0.99996662  0.24925557  0.27699298]
label A min:  [ -1.35587207e+03  -1.29241923e+03   4.90573891e-35]
label A max:  [ -3.77586549e+01  -5.08476984e-30   9.94619004e-01]
label B min:  [ -5.45260155e-09   4.76076392e-01]
label B max:  [ 13.19452088   1.        ]
label P min:  [ -1.20215621e+01   1.28077084e-37]
label P max:  [ 0.00755388  0.24925554]
label S min:  [ -1.72322236e+00   9.88635155e-43]
label S max:  [ -1.11352039e-37   2.76989729e-01]


In [20]:
model_dict["n_inputs"] = 4

modelGrad_s = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_gradA_s.pt"
modelGrad_s.load_state_dict(torch.load(model_path))
modelGrad_s.eval()

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_gradA_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_A_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_A_model = A_Model(modelGrad_s, modelGrad_f, model_f,
                       scaler_X_ABP.data_max_, scaler_X_ABP.data_min_,
                       scaler_y_A.data_max_, scaler_y_A.data_min_)

X = data[["dist", "A_av", "B_av", "P_av"]].values
model_trace = torch.jit.trace(full_A_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "A_model_cons.pt")

In [21]:
# sanity test
A_model_cons = torch.jit.load(output_path_model + "A_model_cons.pt")
X = data[["dist", "A_av", "B_av", "P_av"]].values
y = data[["gradA_s", "gradA_f", "A_f"]].values
ym = A_model_cons(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[  1.06967755e+02   1.02455355e+02   8.99112268e-03]


In [22]:
model_dict["n_inputs"] = 4

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_gradB_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_B_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_B_model = BPS_Model(modelGrad_f, model_f,
                         scaler_X_ABP.data_max_, scaler_X_ABP.data_min_,
                         scaler_y_B.data_max_, scaler_y_B.data_min_)

X = data[["dist", "A_av", "B_av", "P_av"]].values
model_trace = torch.jit.trace(full_B_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "B_model_cons.pt")

In [23]:
# sanity test
B_model_cons = torch.jit.load(output_path_model + "B_model_cons.pt")
X = data[["dist", "A_av", "B_av", "P_av"]].values
y = data[["gradB_f", "B_f"]].values
ym = B_model_cons(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[ 0.58645418  0.0124481 ]


In [24]:
model_dict["n_inputs"] = 4

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_gradP_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_P_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_P_model = BPS_Model(modelGrad_f, model_f,
                         scaler_X_ABP.data_max_, scaler_X_ABP.data_min_,
                         scaler_y_P.data_max_, scaler_y_P.data_min_)

X = data[["dist", "A_av", "B_av", "P_av"]].values
model_trace = torch.jit.trace(full_P_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "P_model_cons.pt")

In [25]:
# sanity test
P_model_cons = torch.jit.load(output_path_model + "P_model_cons.pt")
X = data[["dist", "A_av", "B_av", "P_av"]].values
y = data[["gradP_f", "P_f"]].values
ym = P_model_cons(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[ 1.19858018  0.00173992]


In [26]:
model_dict["n_inputs"] = 5

modelGrad_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_gradS_f.pt"
modelGrad_f.load_state_dict(torch.load(model_path))
modelGrad_f.eval()

model_f = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_S_f.pt"
model_f.load_state_dict(torch.load(model_path))
model_f.eval()

full_S_model = BPS_Model(modelGrad_f, model_f,
                         scaler_X_ABPS.data_max_, scaler_X_ABPS.data_min_,
                         scaler_y_S.data_max_, scaler_y_S.data_min_)

X = data[["dist", "A_av", "B_av", "P_av", "S_av"]].values
model_trace = torch.jit.trace(full_S_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "S_model_cons.pt")

In [27]:
# sanity test
S_model_cons = torch.jit.load(output_path_model + "S_model_cons.pt")
X = data[["dist", "A_av", "B_av", "P_av", "S_av"]].values
y = data[["gradS_f", "S_f"]].values
ym = S_model_cons(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

[ 0.1011508   0.00194043]


## Source terms
### Single reaction

In [28]:
data = data_files["single"]
scaler_X_rAB = MinMaxScaler()
scaler_y_rAB = MinMaxScaler()
X01_rAB = scaler_X_rAB.fit_transform(data[["dist", "A_av", "B_av"]].values)
y01_rAB = scaler_y_rAB.fit_transform(data[["rAB_av"]].values)
print("X rAB min: ", scaler_X_rAB.data_min_)
print("X rAB max: ", scaler_X_rAB.data_max_)
print("y rAB min: ", scaler_y_rAB.data_min_)
print("y rAB max: ", scaler_y_rAB.data_max_)

X rAB min:  [  1.46467255e-04   1.90126350e-02   4.70975454e-01]
X rAB max:  [ 0.02410139  0.99859163  0.99996662]
y rAB min:  [ 0.01899498]
y rAB max:  [ 0.96005602]


In [29]:
model_dict["n_inputs"] = 3

model_r = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "single_rAB_av.pt"
model_r.load_state_dict(torch.load(model_path))
model_r.eval()

full_R_model = R_Model(model_r,
                       scaler_X_rAB.data_max_, scaler_X_rAB.data_min_,
                       scaler_y_rAB.data_max_, scaler_y_rAB.data_min_)

X = data[["dist", "A_av", "B_av"]].values
model_trace = torch.jit.trace(full_R_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "rAB_model_single.pt")

In [30]:
# sanity test
rAB_model_single = torch.jit.load(output_path_model + "rAB_model_single.pt")
X = data[["dist", "A_av", "B_av"]].values
y = data[["rAB_av"]].values[:,0]
ym = rAB_model_single(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

0.00648585085187


### Consecutive reaction

In [31]:
data = data_files["cons"]
scaler_X_rAB = MinMaxScaler()
scaler_y_rAB = MinMaxScaler()
X01_rAB = scaler_X_rAB.fit_transform(data[["dist", "A_av", "B_av"]].values)
y01_rAB = scaler_y_rAB.fit_transform(data[["rAB_av"]].values)
print("X rAB min: ", scaler_X_rAB.data_min_)
print("X rAB max: ", scaler_X_rAB.data_max_)
print("y rAB min: ", scaler_y_rAB.data_min_)
print("y rAB max: ", scaler_y_rAB.data_max_)

X rAB min:  [  1.46467255e-04   1.90126183e-02   4.76073262e-01]
X rAB max:  [ 0.02410139  0.99723356  0.99996662]
y rAB min:  [ 0.01899496]
y rAB max:  [ 0.96003565]


In [32]:
model_dict["n_inputs"] = 3

model_r = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_rAB_av.pt"
model_r.load_state_dict(torch.load(model_path))
model_r.eval()

full_R_model = R_Model(model_r,
                       scaler_X_rAB.data_max_, scaler_X_rAB.data_min_,
                       scaler_y_rAB.data_max_, scaler_y_rAB.data_min_)

X = data[["dist", "A_av", "B_av"]].values
model_trace = torch.jit.trace(full_R_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "rAB_model_cons.pt")

In [33]:
# sanity test
rAB_model_cons = torch.jit.load(output_path_model + "rAB_model_cons.pt")
X = data[["dist", "A_av", "B_av"]].values
y = data[["rAB_av"]].values[:,0]
ym = rAB_model_cons(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

0.00536666522764


In [34]:
data = data_files["cons"]
scaler_X_rAP = MinMaxScaler()
scaler_y_rAP = MinMaxScaler()
X01_rAP = scaler_X_rAP.fit_transform(data[["dist", "A_av", "P_av"]].values)
y01_rAP = scaler_y_rAP.fit_transform(data[["rAP_av"]].values)
print("X rAP min: ", scaler_X_rAP.data_min_)
print("X rAP max: ", scaler_X_rAP.data_max_)
print("y rAP min: ", scaler_y_rAP.data_min_)
print("y rAP max: ", scaler_y_rAP.data_max_)

X rAP min:  [  1.46467255e-04   1.90126183e-02   3.33430253e-05]
X rAP max:  [ 0.02410139  0.99723356  0.24925557]
y rAP min:  [  1.76337889e-05]
y rAP max:  [ 0.24854635]


In [35]:
model_dict["n_inputs"] = 3

model_r = hm.SimpleMLP(**model_dict)
model_path = output_path_model + "cons_rAP_av.pt"
model_r.load_state_dict(torch.load(model_path))
model_r.eval()

full_R_model = R_Model(model_r,
                       scaler_X_rAP.data_max_, scaler_X_rAP.data_min_,
                       scaler_y_rAP.data_max_, scaler_y_rAP.data_min_)

X = data[["dist", "A_av", "P_av"]].values
model_trace = torch.jit.trace(full_R_model, torch.from_numpy(X[0]).unsqueeze(-1).T)
model_trace.save(output_path_model + "rAP_model_cons.pt")

In [36]:
# sanity test
rAP_model_cons = torch.jit.load(output_path_model + "rAP_model_cons.pt")
X = data[["dist", "A_av", "P_av"]].values
y = data[["rAP_av"]].values[:,0]
ym = rAP_model_cons(torch.from_numpy(X)).detach().numpy()
diff = ym - y
print(np.amax(diff, axis=0))

0.00176690606511
